In [ ]:
!pip install numpy
!pip install scikit-learn
!pip install pandas
!pip install matplotlib
!pip install umap-learn[plot]
!pip install gensim
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 36.3 MB/s eta 0:00:00


In [ ]:
from statistics import linear_regression
from sklearn import svm
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelBinarizer


from sklearn.pipeline import Pipeline
import nltk
import umap
from nltk.help import upenn_tagset
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, precision_score, average_precision_score,roc_auc_score
import pandas as pd
import numpy as np
import time
import os
from sklearn.metrics import precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import auc
from sklearn.naive_bayes import MultinomialNB

from nltk.corpus import wordnet,subjectivity,stopwords
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tag.perceptron import PerceptronTagger
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support
import re
import string
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression

import torch # torch provides basic functions, from setting a random seed (for reproducability) to creating tensors.
import torch.nn as nn # torch.nn allows us to create a neural network.
import torch.nn.functional as F # nn.functional give us access to the activation and loss functions.
from torch.optim import Adam # optim contains many optimizers. Here, we're using SGD, stochastic gradient descent, specifically Adam.
from torch.utils.data import TensorDataset, DataLoader



import matplotlib.pyplot as plt ## matplotlib allows us to draw graphs.
import seaborn as sns ## seaborn makes it easier to draw nice-looking graphs.

# Set the device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

nltk.download("subjectivity")
nltk.download('tagsets_json')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

cpu


[nltk_data] Downloading package subjectivity to /root/nltk_data...
[nltk_data]   Unzipping corpora/subjectivity.zip.
[nltk_data] Downloading package tagsets_json to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets_json.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [ ]:
stop_words = set(stopwords.words("english"))
def tokenize_samples(samples):
    tokenized_samples = []
    for sample in samples:
        tokens = []
        # Split text into sentences
        sentences = sent_tokenize(sample)
        for sent in sentences:
            # Tokenize each sentence into words
            words = word_tokenize(sent)
            for word in words:
                # Filter out stopwords and unwanted tokens
                if '\n' in word or "\t" in word or "--" in word or "*" in word or word.lower() in stop_words:
                    continue
                if word.strip():
                    # Process the token and add to list
                    tokens.append(word.replace('"', "'").strip().lower())
        tokenized_samples.append(tokens)

    return tokenized_samples

In [ ]:
def prepare_data_of_subj_obj():
    print(subjectivity.categories())
# print(subjectivity.sents(categories="subj"))
    subjective_sentences = subjectivity.sents(categories="subj")
    objective_sentences = subjectivity.sents(categories="obj")

    data_with_labels = []
    assert(len(subjective_sentences) == len(objective_sentences))
    for i in range(len(subjective_sentences)):
        obj_sentence = objective_sentences[i]
        subj_sentence = subjective_sentences[i]
        data_with_labels.append({"label":"obj","sentence":obj_sentence})
        data_with_labels.append({"label":"subj","sentence":subj_sentence})

    # print(upenn_tagset()) -> Μέρη του λόγου σε προτάσεις και σε τι κλήση κλπ βρίσκονται
    lemmatizer = WordNetLemmatizer()
    tagger = PerceptronTagger()
    docs = []
    labels = []
    for doc in tqdm(data_with_labels):
        sentence = " ".join(doc["sentence"])
        label = doc["label"]
        document = re.sub(r'\W', ' ', str(sentence))

        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        document = re.sub(r'\s+', ' ', document, flags=re.I)

        document = document.lower()

        document = document.split()

        doc_pos = [x[1] for x in tagger.tag(document)]

        document = \
            [lemmatizer.lemmatize(token, pos=nltk_pos_tagger(pos_tag)) \
                if nltk_pos_tagger(pos_tag) != None else lemmatizer.lemmatize(token) \
            for token, pos_tag in zip(document, doc_pos)]

        document = ' '.join(document)

        docs.append(document)
        value_of_label = 0 if label == "subj" else 1
        labels.append(value_of_label)
        # labels.append(label)
    # print(docs)
    print(len(subjective_sentences),len(objective_sentences))
    return docs,labels

In [ ]:
docs,labels = prepare_data_of_subj_obj()
X_train_val,X_test,y_train_val,y_test = train_test_split(docs, labels, test_size=0.3, random_state=1924)
X_train,X_val,y_train,y_val = train_test_split(X_train_val,y_train_val,test_size=0.2,random_state=1924)

print('Train samples: {}'.format(len(X_train)))
print('Val samples: {}'.format(len(X_val)))
print('Test samples: {}'.format(len(X_test)))

X_train_tokenized = tokenize_samples(X_train)
X_val_tokenized = tokenize_samples(X_val)
X_test_tokenized = tokenize_samples(X_test)



['obj', 'subj']


100%|██████████| 10000/10000 [00:45<00:00, 221.40it/s]


5000 5000
Train samples: 5600
Val samples: 1400
Test samples: 3000


In [ ]:
def svd_feature_extraction_from_data(train,val,test):

    vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features = 5000, sublinear_tf=True)

    X_train_tfidf = vectorizer.fit_transform([" ".join(x) for x in X_train_tokenized])
    X_val_tfidf = vectorizer.transform([" ".join(x) for x in X_val_tokenized])
    X_test_tfidf = vectorizer.transform([" ".join(x) for x in X_test_tokenized])

    # we use SVD to not change our MLP tf_idf model
    svd = TruncatedSVD(n_components=300, random_state=4321)
    X_train_svd = svd.fit_transform(X_train_tfidf)
    X_val_svd = svd.transform(X_val_tfidf)
    X_test_svd = svd.transform(X_test_tfidf)

    print(X_train_tfidf.shape)
    print(X_val_tfidf.shape)
    print(X_test_tfidf.shape)


    return X_train_svd,X_val_svd,X_test_svd

In [ ]:
X_train_svd,X_val_svd,X_test_svd = svd_feature_extraction_from_data(X_train_tokenized,X_val_tokenized,X_test_tokenized)
print(X_train_svd.shape)
print(X_val_svd.shape)
print(X_test_svd.shape)

(5600, 5000)
(1400, 5000)
(3000, 5000)
(5600, 300)
(1400, 300)
(3000, 300)


In [ ]:
def plot_training_history(history):
    # Plotting loss
    plt.figure()
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plotting accuracy
    plt.figure()
    plt.plot(history['train_accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # Plotting F1 score
    plt.figure()
    plt.plot(history['train_f1_score'], label='Training F1 Score')
    plt.plot(history['val_f1_score'], label='Validation F1 Score')
    plt.title('F1 Score Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.show()


In [ ]:
def plot_training_histories(histories,hyper_parameters):
    # global hyper_parameters
    # Plotting loss
    colors = [("lightgreen","green"),("lightblue","blue"),("pink","red")]
    plt.figure()
    for (index,history) in enumerate(histories):
      plt.plot(history['train_loss'], label=f'Training Loss {hyper_parameters[index][0],hyper_parameters[index][1],hyper_parameters[index][2]}',color=colors[index][0])
      plt.plot(history['val_loss'], label=f'Validation Loss {hyper_parameters[index][0],hyper_parameters[index][1],hyper_parameters[index][2]}',color=colors[index][1])

    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc=(0.42,0.2))
    plt.show()

    plt.figure()
    for (index,history) in enumerate(histories):
        plt.plot(history['train_accuracy'], label=f'Training Accuracy {hyper_parameters[index][0],hyper_parameters[index][1],hyper_parameters[index][2]}',color=colors[index][0])
        plt.plot(history['val_accuracy'], label=f'Validation Accuracy {hyper_parameters[index][0],hyper_parameters[index][1],hyper_parameters[index][2]}',color=colors[index][1])
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    plt.figure()
    for (index,history) in enumerate(histories):
        plt.plot(history['train_f1_score'], label=f'Training Accuracy {hyper_parameters[index][0],hyper_parameters[index][1],hyper_parameters[index][2]}',color=colors[index][0])
        plt.plot(history['val_f1_score'], label=f'Validation Accuracy {hyper_parameters[index][0],hyper_parameters[index][1],hyper_parameters[index][2]}',color=colors[index][1])
    plt.title('F1 Score Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.show()

In [ ]:
import gensim.downloader as api
word2vec = api.load('word2vec-google-news-300')

[=====---------------------------------------------] 10.4% 172.8/1662.8MB downloaded

ContentTooShortError: <urlopen error retrieval incomplete: got only 181137670 out of 1743563840 bytes>

In [ ]:
from collections import Counter


MAX_SEQUENCE_LENGTH = 100
MAX_TOKENS_FOR_VOCABULARY = 100000

# Build vocabulary from tokens in X_train_tokenized
all_tokens = [token for sentence in X_train_tokenized for token in sentence]
token_counts = Counter(all_tokens)

# Initialize vocabulary with special tokens <pad> and <unk>
vocab = {"<pad>": 0, "<unk>": 1}

# Sort tokens by frequency in descending order and add to vocab starting from 2
for i, (token, _) in enumerate(token_counts.most_common(MAX_TOKENS_FOR_VOCABULARY), start=2):
    vocab[token] = i

# Encode and pad each sentence
def encode_sentence(sentence, vocab, max_length=100):
    # Map each token to its vocabulary ID or <unk> if not in vocab
    encoded = [vocab.get(token, vocab["<unk>"]) for token in sentence]
    # Pad or truncate to max_length
    return encoded[:max_length] + [vocab["<pad>"]] * max(0, max_length - len(encoded))

# Apply encoding and padding to each sentence in X_train_tokenized
train_data = [encode_sentence(sentence, vocab, MAX_SEQUENCE_LENGTH) for sentence in X_train_tokenized]
val_data = [encode_sentence(sentence, vocab, MAX_SEQUENCE_LENGTH) for sentence in X_val_tokenized]
test_data = [encode_sentence(sentence, vocab, MAX_SEQUENCE_LENGTH) for sentence in X_test_tokenized]

In [ ]:
EMBEDDING_DIM = 300
# from our data it probably does not need more than 1.5k
MAX_WORDS = 100_000
MAX_SEQUENCE_LENGTH = 300
embedding_matrix = np.zeros((MAX_WORDS+2, EMBEDDING_DIM))

count = 0
for word, i in vocab.items():
    if i > MAX_WORDS:
            continue
    try:
        embedding_matrix[i] = word2vec[word]
        count+=1
    except:
        pass

#Return the number of the elements that are non-zero.
print(len(np.unique(np.nonzero(embedding_matrix)[0])))
print(count)

print(embedding_matrix.shape)

In [ ]:
print(y_train[:5],y_val[:5],y_test[:5])

In [ ]:
train_dataset = TensorDataset(torch.tensor(train_data, dtype = torch.long), torch.tensor(y_train, dtype = torch.long))
val_dataset = TensorDataset(torch.tensor(val_data, dtype = torch.long), torch.tensor(y_val, dtype = torch.long))
test_dataset = TensorDataset(torch.tensor(test_data, dtype = torch.long), torch.tensor(y_test, dtype = torch.long))

BATCH_SIZE = 256
# Create DataLoader for each dataset
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
# _,labels_of_data = prepare_data_of_subj_obj()
print(train_dataset[:5])

In [ ]:
# Hyper-Prameters
FILTERS = 128  # the dimensionality of the output space (i.e. the number of output filters in the convolution)
KERNEL = 3  # the length of the 1D convolution window, in NLP tasks it is looking at Trigrams
DENSE = 128

N_CLASSES = 2
EMBEDDING_DIM = 300
MAX_WORDS = 100_000
MAX_SEQUENCE_LENGTH = 250

class Stacked_CNN(nn.Module):
    def __init__(self, input_dim, n_classes, dense, filters, kernel, matrix_embeddings = None,dropout=0.2):
        super(Stacked_CNN, self).__init__()

        # dimensions
        self.input_dim = input_dim
        self.n_classes = n_classes
        self.filters = filters
        self.kernel = kernel
        self.dense = dense

        # Embedding layer
        if matrix_embeddings is not None:
            self.embedding_layer = nn.Embedding(num_embeddings = MAX_WORDS + 2, embedding_dim = EMBEDDING_DIM, padding_idx=0).from_pretrained(matrix_embeddings)
            self.embedding_layer.requires_grad = False # Don't change the embedding matrix
        else:
            self.embedding_layer = nn.Embedding(num_embeddings = MAX_WORDS + 2, embedding_dim = EMBEDDING_DIM, padding_idx=0)
            self.embedding_layer.requires_grad = True # Learn the embedding matrix

        self.conv1 = nn.Conv1d(in_channels = self.input_dim, out_channels=self.filters, kernel_size=self.kernel, padding = "valid") # Padding can either be "valid" or "same". "same" ensures the output length remains the same
        self.conv2 = nn.Conv1d(in_channels = self.filters, out_channels=self.filters, kernel_size=self.kernel, padding = "valid")   # by padding the input whereas "valid" doesn't pad the input hence reducing the output
        self.conv3 = nn.Conv1d(in_channels = self.filters, out_channels=self.filters, kernel_size=self.kernel, padding = "valid")   # length (the amount it reduces it by is dependent on the kernel size)
        self.conv4 = nn.Conv1d(in_channels = self.filters, out_channels=self.filters, kernel_size=self.kernel, padding = "valid")
        self.conv5 = nn.Conv1d(in_channels = self.filters, out_channels=self.filters, kernel_size=self.kernel, padding = "valid")

        self.global_max_pool = nn.AdaptiveMaxPool1d(output_size = 1)

        self.dense_layer = nn.Linear(in_features = self.filters, out_features = self.dense)

        self.output = nn.Linear(in_features = self.dense, out_features = self.n_classes)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        # Input shape: (batch_size, sequence_length)
        # Embedding layer
        x = self.embedding_layer(input.long()) # (batch_size, sequence_length, embedding_dim)

        # Dropout embeddings
        x = self.dropout(x) # (batch_size, sequence_length, embedding_dim)

        # Transpose x because Conv1d expects input of (batch_size, in_channels, sequence_length) where in_channels maps to our embedding_dim here
        x = x.transpose(1, 2) # (batch_size, embedding_dim, sequence_length)

        # Stack of convolutions. Note that my shapes here are specific to my kernel size which is 3. I explain where these numbers come from in the comments.
        x = self.conv1(x) # (batch_size, filters, sequence_length - 2)             The 2 here comes from kernel_size - 1 where 1 is the stride. This number changes depending on the kernel_size
        x = F.relu(x) # (batch_size, filters, sequence_length - 2)

        x = self.conv2(x) # (batch_size, filters, sequence_length - 4)
        x = F.relu(x) # (batch_size, filters, sequence_length - 4)

        x = self.conv3(x) # (batch_size, filters, sequence_length - 6)
        x = F.relu(x) # (batch_size, filters, sequence_length - 6)

        x = self.conv4(x) # (batch_size, filters, sequence_length - 8)
        x = F.relu(x) # (batch_size, filters, sequence_length - 8)

        x = self.conv5(x) # (batch_size, filters, sequence_length - 10)
        x = F.relu(x) # (batch_size, filters, sequence_length - 10)

        # Global max pooling
        x = self.global_max_pool(x) # (batch_size, filters, 1)
        x = x.squeeze(2) # (batch_size, filters)

        # Dense layer
        x = self.dense_layer(x) # (batch_size, dense)
        x = F.relu(x) # (batch_size, dense)

        # Dropout
        x = self.dropout(x) # (batch_size, dense)

        # Output layer (Logits)
        logits = self.output(x) # (batch_size, n_classes)

        return logits

In [ ]:
stacked_cnn = Stacked_CNN(input_dim = EMBEDDING_DIM,
                          n_classes = N_CLASSES,
                          dense = DENSE,
                          filters = FILTERS,
                          kernel = KERNEL,
                          matrix_embeddings = torch.tensor(embedding_matrix, dtype = torch.float))

print(stacked_cnn)

print(f"Total learnable parameters: {sum(p.numel() for p in stacked_cnn.parameters() if p.requires_grad)}")

In [ ]:
import time
import os

from torchmetrics.classification import MultilabelAccuracy, MultilabelF1Score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model,epochs):
    torch.manual_seed(1)
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Initialize TorchMetrics for multi-label metrics
    # accuracy_metric = MultilabelAccuracy(num_labels=N_CLASSES, threshold=0.5, average='micro').to(device)
    # f1_metric = MultilabelF1Score(num_labels=N_CLASSES, threshold=0.5, average='micro').to(device)

    # Hyperparameters
    best_val_loss = float("inf")
    best_model_path = None
    early_stopping_patience = 5
    early_stopping_counter = 0

    # Training history
    history = {
        'train_loss': [], 'val_loss': [],
        'train_accuracy': [], 'val_accuracy': [],
        'train_f1_score': [], 'val_f1_score': [],
        'train_roc_auc':[], 'val_roc_auc': []
    }

    start_training_time = time.time()

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        train_outputs = []
        train_labels = []

        # Training phase
        for batch in train_loader:
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(features)

            # Loss calculation
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            train_outputs.append(outputs)
            train_labels.append(labels)

        # Post-epoch training calculations
        train_outputs = torch.cat(train_outputs)
        train_labels = torch.cat(train_labels)

        # Convert logits to binary predictions
        train_predicted_classes = torch.argmax(train_outputs, dim=1)
        # train_predicted_classes = (train_outputs.sigmoid() > 0.5).float()

        # Compute metrics using TorchMetrics
        train_accuracy = accuracy_score(train_predicted_classes, train_labels)
        train_f1_score = f1_score(train_predicted_classes, train_labels)
        train_auc = roc_auc_score(train_predicted_classes,train_labels)

        train_loss = running_loss / len(train_loader)
        history['train_loss'].append(train_loss)
        history['train_accuracy'].append(train_accuracy)
        history['train_f1_score'].append(train_f1_score)
        history['train_roc_auc'].append(train_auc)

        print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, F1 Score: {train_f1_score:.4f}")

        # Validation phase
        model.eval()
        val_running_loss = 0.0
        val_outputs = []
        val_labels = []

        with torch.no_grad():
            for features, labels in val_loader:
                features = features.to(device)
                labels = labels.to(device)

                outputs = model(features)

                # Validation loss
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                val_outputs.append(outputs)
                val_labels.append(labels)

        # Post-epoch validation calculations
        val_outputs = torch.cat(val_outputs)
        val_labels = torch.cat(val_labels)

        # Convert logits to binary predictions
        val_predicted_classes = torch.argmax(val_outputs,dim=1)
        # val_predicted_classes = (val_outputs.sigmoid() > 0.5).float()

        # Compute validation metrics using TorchMetrics
        val_accuracy = accuracy_score(val_predicted_classes, val_labels)
        val_f1 = f1_score(val_predicted_classes, val_labels)
        val_auc = roc_auc_score(val_predicted_classes,val_labels)

        val_loss = val_running_loss / len(val_loader)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)
        history['val_f1_score'].append(val_f1)
        history['val_roc_auc'].append(val_auc)

        print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, F1 Score: {val_f1:.4f}")

        # Early stopping and checkpointing
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0  # Reset counter if validation improves

            new_model_path = f'model_best_epoch_{epoch + 1}.pth'

            # Delete the previous best model
            if best_model_path is not None:
                os.remove(best_model_path)

            # Save the new best model
            torch.save(model.state_dict(), new_model_path)
            best_model_path = new_model_path
        else:
            early_stopping_counter += 1

        # Stop training if no improvement for 5 epochs
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered. No improvement in validation loss for 5 consecutive epochs.")
            break

    print("Training complete!")
    end_training_time = time.time()
    return model,history,best_model_path




In [ ]:
print(embedding_matrix[:5])

In [ ]:
hyper_parameters = [(DENSE,FILTERS,KERNEL)]
cnn_results = []
for dense,filters,kernel in hyper_parameters:
    model = Stacked_CNN(input_dim = EMBEDDING_DIM,
                            n_classes = N_CLASSES,
                            dense = dense,
                            filters = filters,
                            kernel = kernel,
                            matrix_embeddings = torch.tensor(embedding_matrix, dtype = torch.float))
    result = train_model(model,5)
    cnn_results.append(result)

In [ ]:
plot_training_histories(list(map(lambda x: x[1],cnn_results)),hyper_parameters)

In [ ]:
for pos,(model,history,best_model_path) in enumerate(cnn_results):
    model.load_state_dict(torch.load(best_model_path, weights_only = True))
    dense,filters,kernel = hyper_parameters[pos]
    pred_training = model(torch.tensor(train_data,dtype=torch.long))
    y_pred_training = torch.argmax(pred_training,dim=1)
    print(model)
    model_description = f"Stacked CNN with Dense: {dense} filters: {filters} kernels {kernel}"
    print(f"Results in Training Data for {model_description}")
    print(classification_report(y_train, y_pred_training, target_names=["subj","obj"]))

    pred_val = model(torch.tensor(val_data,dtype=torch.long))
    y_pred_val = torch.argmax(pred_val, dim=1)
    print(model)
    print(f"Results in Validation Data for {model_description}")
    print(classification_report(y_val, y_pred_val, target_names=["subj","obj"]))

    print(f'{model_description} Validation Accuracy: {accuracy_score(y_val, y_pred_val)*100:.2f}%')

    predictions_test = model(torch.tensor(test_data,dtype=torch.long))
    y_pred_test = torch.argmax(predictions_test, dim=1)
    print(f'{model_description} Test Accuracy:{accuracy_score(y_test, y_pred_test)*100:.2f}%')
    print(classification_report(y_test, y_pred_test, target_names=["subj","obj"]))